In [1]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats
import os
import rasterio

from tqdm import tqdm

import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np

import re

In [2]:
df = pd.read_csv(r"C:\Users\jarem\OneDrive - London School of Economics\YEAR 2\PP4V8 - policy paper\policy-paper-repo\data\inputs\EUSTAT ARDECO\SUVGD - MIO_EUR - GDP at current market prices - NUTS_2021.csv")

In [3]:
# mapping dicts: iso2 to iso3
iso2_to_iso3 = {
    "AF": "AFG", "AL": "ALB", "DZ": "DZA", "AS": "ASM", "AD": "AND",
    "AO": "AGO", "AI": "AIA", "AQ": "ATA", "AG": "ATG", "AR": "ARG",
    "AM": "ARM", "AW": "ABW", "AU": "AUS", "AT": "AUT", "AZ": "AZE",
    "BS": "BHS", "BH": "BHR", "BD": "BGD", "BB": "BRB", "BY": "BLR",
    "BE": "BEL", "BZ": "BLZ", "BJ": "BEN", "BM": "BMU", "BT": "BTN",
    "BO": "BOL", "BA": "BIH", "BW": "BWA", "BR": "BRA", "BN": "BRN",
    "BG": "BGR", "BF": "BFA", "BI": "BDI", "KH": "KHM", "CM": "CMR",
    "CA": "CAN", "CV": "CPV", "KY": "CYM", "CF": "CAF", "TD": "TCD",
    "CL": "CHL", "CN": "CHN", "CO": "COL", "KM": "COM", "CG": "COG",
    "CD": "COD", "CR": "CRI", "HR": "HRV", "CU": "CUB", "CY": "CYP",
    "CZ": "CZE", "DK": "DNK", "DJ": "DJI", "DO": "DOM", "EC": "ECU",
    "EG": "EGY", "SV": "SLV", "GQ": "GNQ", "ER": "ERI", "EE": "EST",
    "ET": "ETH", "FI": "FIN", "FR": "FRA", "GA": "GAB", "GM": "GMB",
    "GE": "GEO", "DE": "DEU", "GH": "GHA", "GR": "GRC", "GL": "GRL",
    "GT": "GTM", "GN": "GIN", "GW": "GNB", "GY": "GUY", "HT": "HTI",
    "HN": "HND", "HK": "HKG", "HU": "HUN", "IS": "ISL", "IN": "IND",
    "ID": "IDN", "IR": "IRN", "IQ": "IRQ", "IE": "IRL", "IL": "ISR",
    "IT": "ITA", "JM": "JAM", "JP": "JPN", "JO": "JOR", "KZ": "KAZ",
    "KE": "KEN", "KI": "KIR", "KP": "PRK", "KR": "KOR", "KW": "KWT",
    "KG": "KGZ", "LA": "LAO", "LV": "LVA", "LB": "LBN", "LS": "LSO",
    "LR": "LBR", "LY": "LBY", "LT": "LTU", "LU": "LUX", "MO": "MAC",
    "MG": "MDG", "MW": "MWI", "MY": "MYS", "MV": "MDV", "ML": "MLI",
    "MT": "MLT", "MX": "MEX", "MD": "MDA", "MC": "MCO", "MN": "MNG",
    "ME": "MNE", "MA": "MAR", "MZ": "MOZ", "MM": "MMR", "NA": "NAM",
    "NP": "NPL", "NL": "NLD", "NZ": "NZL", "NI": "NIC", "NE": "NER",
    "NG": "NGA", "NO": "NOR", "OM": "OMN", "PK": "PAK", "PA": "PAN",
    "PG": "PNG", "PY": "PRY", "PE": "PER", "PH": "PHL", "PL": "POL",
    "PT": "PRT", "QA": "QAT", "RO": "ROU", "RU": "RUS", "RW": "RWA",
    "SA": "SAU", "SN": "SEN", "RS": "SRB", "SG": "SGP", "SK": "SVK",
    "SI": "SVN", "ZA": "ZAF", "ES": "ESP", "LK": "LKA", "SD": "SDN",
    "SE": "SWE", "CH": "CHE", "SY": "SYR", "TW": "TWN", "TJ": "TJK",
    "TZ": "TZA", "TH": "THA", "TN": "TUN", "TR": "TUR", "UA": "UKR",
    "AE": "ARE", "GB": "GBR", "US": "USA", "UY": "URY", "UZ": "UZB",
    "VE": "VEN", "VN": "VNM", "YE": "YEM", "ZM": "ZMB", "ZW": "ZWE"
}

In [4]:
df["iso3"] = df["NUTS"].str[:2].map(iso2_to_iso3)

In [11]:
# filter to CEE regions

target_iso3 = ['POL', 'CZE', 'SVK', 'HUN', 'EST', 'LVA', 'LTU', 'ROU', 'BGR', 'HRV', 'SVN']
df_filtered = df[df['iso3'].isin(target_iso3)]

df_filtered.dropna(axis=0, how='all')

,NUTS,Level,NUTS Version,Unit,1980,1981,1982,1983,1984,1985,...,2019,2020,2021,2022,2023,2024,2025,2026,2027,iso3
83,BG,0,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,61195.40,61856.50,71345.00,86079.50,94526.60,104768.80,114182.02,121344.32,126973.42,BGR
84,BG311,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,373.94,370.59,463.59,495.19,555.78,616.00,671.35,713.46,746.56,BGR
85,BG312,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,623.89,659.70,750.63,767.81,890.45,986.93,1075.60,1143.07,1196.10,BGR
86,BG313,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,1092.41,1267.88,1397.72,2034.54,2051.18,2273.43,2477.69,2633.11,2755.26,BGR
87,BG314,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,1194.46,1218.90,1507.54,1804.64,1979.50,2193.98,2391.10,2541.09,2658.97,BGR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,SK031,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,10731.20,10405.22,11489.40,12502.63,13736.72,14478.25,15152.82,15847.13,16533.61,SVK
1216,SK032,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,8113.03,8023.68,9047.99,9987.94,10838.72,11423.81,11956.07,12503.90,13045.56,SVK
1217,SK041,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,8751.38,8750.33,9326.34,10235.28,11488.28,12108.43,12672.59,13253.25,13827.37,SVK
1218,SK042,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,11033.97,11348.03,12391.52,13386.43,14985.29,15794.22,16530.11,17287.53,18036.41,SVK


In [18]:
df_filtered = df_filtered.loc[:, df_filtered.notna().any()]


In [15]:
years = [int(c) for c in df_filtered.columns if re.fullmatch(r"\d{4}", str(c))]

print(min(years), max(years))


1980 2027


In [23]:
display(df_filtered)

,NUTS,Level,NUTS Version,Unit,1980,1981,1982,1983,1984,1985,...,2019,2020,2021,2022,2023,2024,2025,2026,2027,iso3
83,BG,0,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,61195.40,61856.50,71345.00,86079.50,94526.60,104768.80,114182.02,121344.32,126973.42,BGR
84,BG311,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,373.94,370.59,463.59,495.19,555.78,616.00,671.35,713.46,746.56,BGR
85,BG312,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,623.89,659.70,750.63,767.81,890.45,986.93,1075.60,1143.07,1196.10,BGR
86,BG313,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,1092.41,1267.88,1397.72,2034.54,2051.18,2273.43,2477.69,2633.11,2755.26,BGR
87,BG314,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,1194.46,1218.90,1507.54,1804.64,1979.50,2193.98,2391.10,2541.09,2658.97,BGR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,SK031,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,10731.20,10405.22,11489.40,12502.63,13736.72,14478.25,15152.82,15847.13,16533.61,SVK
1216,SK032,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,8113.03,8023.68,9047.99,9987.94,10838.72,11423.81,11956.07,12503.90,13045.56,SVK
1217,SK041,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,8751.38,8750.33,9326.34,10235.28,11488.28,12108.43,12672.59,13253.25,13827.37,SVK
1218,SK042,3,2021,MIO_EUR,NaN,NaN,NaN,NaN,NaN,NaN,...,11033.97,11348.03,12391.52,13386.43,14985.29,15794.22,16530.11,17287.53,18036.41,SVK
